In [1]:
import geopandas as gpd
import pandas as pd
import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location
from cityiq.scrape import EventScraper

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt



In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [3]:
locations = pkg.resource('locations').geoframe()
assets = pkg.resource('assets').geoframe()

locations.head()

,locationUid,locationType,parentLocationUid,roadsegid,speed,oneway,abloaddr,abhiaddr,rd30full,geometry
0,004361eb,WALKWAY,004361eb,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...
1,00456472,TRAFFIC_LANE,00456472,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1705245131968 32.726558517207...
2,0051796c,WALKWAY,0051796c,54946.0,20.0,F,1600.0,1699.0,STATE ST,LINESTRING (-117.1666581391174 32.722214661862...
3,00537bf3,TRAFFIC_LANE,00537bf3,54946.0,20.0,F,1600.0,1699.0,STATE ST,LINESTRING (-117.166581746989 32.7222178855665...
4,005f90ed,WALKWAY,005f90ed,54945.0,20.0,F,1700.0,1799.0,STATE ST,LINESTRING (-117.1665182431566 32.723810889902...


In [4]:
assets.head()

,assetUid,assetType,parentAssetUid,mediaType,events,roadsegid,speed,oneway,abloaddr,abhiaddr,rd30full,geometry
0,000223ee-a868-474b-abcb-12ff1bad00a3,CAMERA,131d67b7-daaf-4252-9202-5df347a6c2a3,"IMAGE,VIDEO",,1453.0,20.0,F,500.0,599.0,05TH AVE,POINT (-117.1600173 32.71143062)
1,0002e3bb-5a9a-4083-9a96-fad0d22877b9,MIC,e8beafbf-7836-48c3-bb14-440970eecd6d,AUDIO,,3046.0,20.0,B,400.0,499.0,16TH ST,POINT (-117.1493049 32.70997207)
2,0003806e-e409-449f-aab1-fc95f6e88d4e,CAMERA,f91ec557-f03e-4189-8b83-5aa8d28fbf14,"IMAGE,VIDEO",,2270.0,20.0,B,400.0,499.0,09TH AVE,POINT (-117.1566029 32.70934863)
3,000a4ac7-224b-4fb7-a2e2-cd0592d2b2de,CAMERA,741ce488-72fa-4216-a65d-216f7bdf8b7d,"IMAGE,VIDEO",,3152.0,20.0,B,900.0,999.0,17TH ST,POINT (-117.148666 32.71585673)
4,000b2365-5309-422a-9be6-1b7127ca18db,NODE,,,,40722.0,20.0,B,5300.0,5349.0,LA JOLLA HERMOSA AVE,POINT (-117.264282290807 32.8105285177251)


In [5]:
parking = [ e for e in loc_rec if e.locationType == 'PARKING_ZONE']

NameError: name 'loc_rec' is not defined

In [ ]:
import metapack as mp
pkg = mp.open_package('http://library.metatab.org/sandiegodata.org-geography-2018.csv')
pkg

In [ ]:
tracts = pkg.resource('tract_boundaries').geoframe()
tract_comm =  pkg.resource('tract_communities').dataframe()

In [ ]:
t = gpd.GeoDataFrame(
    [ (e.locationUid, e.geometry) for e in parking],
    columns = ['locationUid','geometry'],
    geometry='geometry')
t.crs = tracts.crs
t['geometry'] = t.geometry.centroid

t = gpd.sjoin(t, tracts)
parking_geo = t.merge(tract_comm, on='geoid').drop(columns=['geometry', 'index_right'])
parking_geo.head()

In [ ]:
parking_geo.to_csv('parking_geo.csv')

In [ ]:
com = pkg.resource('sd_community_boundaries').geoframe()
com.plot()

In [ ]:
com = com.to_crs({'init':'epsg:2875'})
com.plot()

In [ ]:
pkg = mp.jupyter.open_package()
roads = pkg.resource('all_roads').geoframe()

sdroads = roads[(roads.lpsjur == 'SD') | (roads.rpsjur == 'SD')].copy()

# Convert to EPSG:2875 ( California zone 6, feet ) so we can make the buffer in feet
sdroads = sdroads.to_crs({'init':'epsg:2875'})

# Buffer the roads to be big enough to enclose the assets
sdroads['geometry'] = sdroads.geometry.apply(lambda g : g.buffer(60,cap_style=1, join_style=2))
